# Mixed Radix Math
We can represent all numbers in the Collatz Problem as a list of tuples to be summed of the form:

$$
((a, b), (numer, denom))
$$

Where:
- $a$ is the power of 2
- $b$ is the power of 3
- $(numer, denom)$ gives the rational weight of the value.

We will call these "Mixed Radix Math" (MRM) tuples since we are already using the term "MRTup"

In [65]:
import copy
import math
from fractions import Fraction
from sympy import factorint

In [136]:
def fractionForMR(a, b):
    """
    Given the power of 2, a; and the power of 3, b; generate the corresponding Fraction
    """
    if a < 0:
        if b < 0:
            return Fraction(1, (2**a)*(3**b))
        else:
            return Fraction((3**b), (2**a))
    else:
        if b < 0:
            return Fraction((2**a), (3**b))
        else:
            return Fraction((2**a)*(3**b), 1)
#        

class MRMTup:
    """
    A class to keep track of Mixed Radix Math that stores Mixed Radix values as a dictionary keyed on the (a, b) tuple.
    """
    def __init__(self, initializer=None):
        if initializer is None:
            self.value = {}
        else:
            if isinstance(initializer, self.__class__): 
                self.value = copy.deepcopy(initializer.value)
            else:
                # Expect a list of ((a, b), (numer, denom)) tups
                self.value = {}
                for tup in initializer:
                    key = tup[0]
                    val = tup[1]
                    self.value[key] = Fraction(*val)
    #
    def evaluate(self):
        result = Fraction(0, 1)
        for key in self.value:
            result = result + (fractionForMR(*key)  * self.value[key])
        return result
    #
    def __add__(self, other):
        result = MRMTup(self)
        for key in other.value:
            if key in self.value:
                val = self.value[key] + other.value[key]
                if val.numerator == 0:
                    del result.value[key]
                else:
                    result.value[key] = val
            else:
                result.value[key] = other.value[key]
        return result
    #
    def __neg__(self):
        result = MRMTup(self)
        for key in result.value:
            result.value[key] = - result.value[key]
        return result
    #
    def __sub__(self, other):
        return self.__add__(- other)
    #
    def __mul__(self, other):
        if isinstance(other, self.__class__):
            result = MRMTup()
            if len(self.value) == 0 or len(other.value) == 0:
                return result
            for key_0 in self.value:
                for key_1 in other.value:
                    key_result = (key_0[0] + key_1[0], key_0[1] + key_1[1])
                    val_result = self.value[key_0] * other.value[key_1]
                    if key_result in result.value:
                        result.value[key_result] = result.value[key_result] + val_result
                    else:
                        result.value[key_result] = val_result
        else:
            """
            Multiply by an integer, adjusting factors of 2,3 in the process
            """
            # TODO: make this work for Fractions
            result = MRMTup(self)
            ival = other
            factors = factorint(ival)
            if 2 in factors:        
                for key in list(result.value.keys()):
                    result.value[(key[0] + factors[2], key[1])] = result.value[key]
                    del result.value[key]
                ival = ival // (2**factors[2])
            if 3 in factors:        
                for key in list(result.value.keys()):
                    result.value[(key[0], key[1] + factors[3])] = result.value[key]
                    del result.value[key]
                ival = ival // (3**factors[3])
                
            if ival != 1:
                for key in list(result.value.keys()):
                    result.value[key] = result.value[key] * ival
            #
        #
        return result
    #
    def __iadd__(self, ival):
        # Works for ival as integer or Fraction
        if (0, 0) in self.value:
            self.value[(0,0)] += ival
            if self.value[(0,0)].numerator == 0:
                del self.value[(0,0)]
        else:
            self.value[(0,0)] = Fraction(ival, 1)
    #
    def __isub__(self, ival):
        # Works for ival as integer or Fraction
        if (0, 0) in self.value:
            self.value[(0,0)] -= ival
            if self.value[(0,0)].numerator == 0:
                del self.value[(0,0)]
        else:
            self.value[(0,0)] = Fraction(-ival, 1)
    #
#

In [127]:
mtmT_1 = MRMTup([((1, 1), (1, 5))])

In [128]:
mtmT_1.value

{(1, 1): Fraction(1, 5)}

In [129]:
(mtmT_1 + mtmT_1).value

{(1, 1): Fraction(2, 5)}

In [130]:
mtmT_1.value

{(1, 1): Fraction(1, 5)}

In [131]:
(mtmT_1 - mtmT_1).value

{}

In [132]:
mtmT_1.evaluate()

Fraction(6, 5)

In [133]:
mtmT_1 *= 210
mtmT_1.value

{(2, 2): Fraction(7, 1)}